In [43]:
import pandas as pd

import requests
import bs4 as bs
from urllib.parse import urljoin
import pickle

%matplotlib inline

# Test (Spain)

https://en.wikipedia.org/wiki/Category:Opinion_polling_in_Spain

In [37]:
#Get all election links

#restrictions in link
restrictions = ["Opinion polling","general election"]
#main url
url = "https://en.wikipedia.org/wiki/Category:Opinion_polling_in_Spain"

#Extract all links
links = []
r = requests.get(url)
if r.ok:
    html = bs.BeautifulSoup(r.text,"html.parser")
    for link in html.find_all("a"):
        if link.has_attr('href') and all([restr in link.text for restr in restrictions]): #check restrictions
            links.append(urljoin(url,link["href"]))
print(links)
                    
                    

['https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1982', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1986', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1989', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1993', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1996', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2000', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2004', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2008', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2011', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2015', 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_2016', 'https://en.wikipedia.org/wiki/

In [38]:
all_tables = []

#position table in website
table_position = 0
for p_url in links:
    #Main ulr
    #p_url = "https://en.wikipedia.org/wiki/Opinion_polling_for_the_Spanish_general_election,_1982"

    #Get columns
    r = requests.get(p_url)
    if r.ok:
        html = bs.BeautifulSoup(r.text,"html.parser")
        tables = html.find_all("table")[table_position]

        rows = tables.find_all("th")
        columns = [r.find("img")["alt"] for r in rows if r.find("img") is not None]

    #Get tables of polls
    polls = pd.read_html(r.text,header=0)[table_position]

    #Rename columns
    types = ["firm","date","error","size","lead"]
    new_cols = []
    for col in polls.columns:
        flag = 0
        for t in types:
            if t in col:
                new_cols.append(t)
                flag = 1
                break
        if flag == 0:
            if "Unnamed" in col:
                new_cols.append(columns.pop(0))
            else:
                new_cols.append(t)

    polls.columns = new_cols
    
    all_tables.append(polls)

In [44]:
pickle.dump(all_tables,open("all_tables.dump","wb+"))

In [36]:
polls.head()

,firm,date,UCD,PSOE,PCE,AP–PDP,CiU,PNV,CDS,error,size,lead
0,General Election,"October 28, 1982",6.8,48.1,4.0,26.4,3.7,1.9,2.9,NaN,NaN,21.7
1,UCD,"October 28, 1982",7.2,45.7,3.8,24.9,4.5,2.1,2.6,NaN,NaN,20.8
2,PSOE,"October 28, 1982",6.6,48.0,4.3,26.0,4.0,2.0,3.1,±0.3 pp,85300.0,22.0
3,ISIS,"October 19, 1982",17.0,45.0,NaN,15,NaN,NaN,NaN,NaN,36792.0,28.0
4,Sofemasa,"October 19, 1982",6.5,48.6,5.9,24.0,2.5,1.7,4.0,±0.7 pp,18255.0,24.6


# Systematical

In [ ]:
#Opinion polling by country
https://en.wikipedia.org/wiki/Category:Opinion_polling_by_country